In [1]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import SpectralEmbedding
import joblib, json, glob

In [2]:
raw_data_root = Path("/nfs2/harmonization/BIDS/MASiVar/derivatives/")
output_root = Path("/home-local/lij112/codes/beyond_fa_challenge/results/Inputs-master")
targets = list(raw_data_root.glob("sub-*/ses-*/PreQual/PREPROCESSED"))
len(targets)

118

In [4]:
def convert_nifti_to_mha(input_file, output_file):
    image = sitk.ReadImage(input_file)
    sitk.WriteImage(image, output_file)


def bval_bvec_to_json(bval_file, bvec_file, output_json_path):
    with open(bval_file, "r") as f:
        bvals = list(map(float, f.readline().strip().split()))
    with open(bvec_file, "r") as f:
        bvecs = [list(map(float, line.strip().split())) for line in f.readlines()]

    data = []
    for i in range(len(bvals)):
        entry = {
            "BVAL": bvals[i] if i < len(bvals) else None,
            "BVEC": [
                bvecs[0][i],
                bvecs[1][i],
                bvecs[2][i],
            ],
        }
        data.append(entry)

    with open(output_json_path, "w") as f:
        json.dump(data, f, indent=4)

    print(f"JSON file saved to {output_json_path}")

In [ ]:
for processed_dir in targets:
    processed_dir = Path(processed_dir)

    sub_id = next(p for p in processed_dir.parents if p.name.startswith("sub-")).name
    ses_id = next(p for p in processed_dir.parents if p.name.startswith("ses-")).name
    tag = f"{sub_id}_{ses_id}"

    nifti = processed_dir / "dwmri.nii.gz"
    bval = processed_dir / "dwmri.bval"
    bvec = processed_dir / "dwmri.bvec"

    if not (nifti.exists() and bval.exists() and bvec.exists()):
        print(f"Lack important file, jump through: {processed_dir}")
        continue

    out_dir = output_root / tag
    out_dir.mkdir(parents=True, exist_ok=True)
    out_mha = out_dir / f"{tag}.mha"
    out_json = out_dir / f"{tag}.json"

    convert_nifti_to_mha(nifti, out_mha)
    bval_bvec_to_json(bval, bvec, out_json)
    print(f"Finish: {tag}")

In [ ]:
x = np.loadtxt(
    "/home-local/lij112/codes/beyond_fa_challenge/results/Stats-csv-master/sub-cIVs044_ses-s1Bx1_pq_dwmri_features.csv",
    delimiter=",",
    skiprows=1,
)
x

array([0.31568466, 0.30538484, 0.15782933, ..., 0.00055266, 0.00069642,
       0.00073239])

In [ ]:
input_pattern = (
    "/home-local/lij112/codes/beyond_fa_challenge/results/Stats-csv-master/*.csv"
)

n_pca = 60
n_spectral = 116
scaler_pca = (
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/scaler_pca.joblib"
)
pca_model = (
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/pca_model.joblib"
)
scaler_spec = (
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/scaler_spec.joblib"
)
spec_model = (
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/spec_model.joblib"
)

paths = sorted(glob.glob(input_pattern))
X = np.vstack([np.loadtxt(p, delimiter=",", skiprows=1)[None, :] for p in paths])
print("X shape:", X.shape)

scaler = StandardScaler().fit(X)
Xs = scaler.transform(X)

pca = PCA(n_components=min(n_pca, Xs.shape[0] - 1, Xs.shape[1]))
Z_pca = pca.fit_transform(Xs)
print("PCA result shape:", Z_pca.shape)
joblib.dump(scaler, scaler_pca)
joblib.dump(pca, pca_model)

spec = SpectralEmbedding(
    n_components=min(n_spectral, Xs.shape[0] - 2, Xs.shape[1]),
    affinity="nearest_neighbors",
    n_neighbors=10,
    random_state=0,
)
Z_spec = spec.fit_transform(Xs)
print("SpectralEmbedding result shape:", Z_spec.shape)
joblib.dump(scaler, scaler_spec)
joblib.dump(spec, spec_model)

X shape: (118, 2592)
PCA result shape: (118, 60)
SpectralEmbedding result shape: (118, 116)


['/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_master/scripts/model/spec_model.joblib']

In [5]:
explained = pca.explained_variance_ratio_.cumsum()
checkpoints = [32, 64, 96, n_pca]
for k in checkpoints:
    idx = min(k, n_pca) - 1
    var = explained[idx]
    print(f"Top {k} PCs explain {var*100:.1f}% of total variance")

Top 32 PCs explain 87.1% of total variance
Top 64 PCs explain 95.1% of total variance
Top 96 PCs explain 95.1% of total variance
Top 60 PCs explain 95.1% of total variance


In [ ]:
scaler_pca = joblib.load(
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/scaler_pca.joblib"
)
pca_model = joblib.load(
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/pca_model.joblib"
)

input_csv = "/home-local/lij112/codes/beyond_fa_challenge/results/Stats-csv-master/sub-cIVs026_ses-s1Bx2_features.csv"
output_json = "/home-local/lij112/codes/beyond_fa_challenge/results/sub-cIVs026_ses-s1Bx2_vector_pca.json"

x = np.loadtxt(
    input_csv,
    delimiter=",",
    skiprows=1,
).reshape(1, -1)

x_scaled = scaler_pca.transform(x)
z = pca_model.transform(x_scaled).flatten()

vec128 = list(z)
if len(vec128) < 128:
    vec128.extend([0.0] * (128 - len(vec128)))

with open(output_json, "w") as f:
    json.dump(vec128, f, indent=4)

print(f"Saved padded 128-D vector to {output_json}")

Saved padded 128-D vector to /home-local/lij112/codes/beyond_fa_challenge/results/sub-cIVs026_ses-s1Bx2_vector_pca.json


In [ ]:
scaler_spec = joblib.load(
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/scaler_spec.joblib"
)
spec_model = joblib.load(
    "/home-local/lij112/codes/beyond_fa_challenge/beyond_fa_pca/scripts/model/spec_model.joblib"
)

input_csv = "/home-local/lij112/codes/beyond_fa_challenge/results/Stats-csv-master/sub-cIVs026_ses-s1Bx2_features.csv"
output_json = "/home-local/lij112/codes/beyond_fa_challenge/results/sub-cIVs026_ses-s1Bx2_vector_spec.json"

X_train = np.vstack([
    np.loadtxt(p, delimiter=",", skiprows=1)[None,:]
    for p in paths
])
Xs_train = scaler_spec.transform(X_train)

x_new = np.loadtxt(input_csv, delimiter=",", skiprows=1).reshape(1, -1)
x_new = scaler_spec.transform(x_new)

X_full = np.vstack([Xs_train, x_new])
Z_full = spec_model.fit_transform(X_full)

z_new = Z_full[-1]
vec   = z_new.tolist()
if len(vec) < 128:
    vec.extend([0.0] * (128 - len(vec)))

with open(output_json, "w") as f:
    json.dump(vec, f, indent=4)

print(f"Saved padded 128-D spectral vector to {output_json}")

Saved padded 128-D spectral vector to /home-local/lij112/codes/beyond_fa_challenge/results/sub-cIVs026_ses-s1Bx2_vector_spec.json
